In [1]:
#!pip install keras-tcn

In [1]:
import pandas as pd
import numpy as np
import random as rnd
import os
# visualization
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

# Preprocessing
from sklearn import preprocessing
import datetime

# machine learning
from keras.layers import Dense, Dropout
from keras.models import Input, Model
from tcn import TCN

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

Using TensorFlow backend.


In [2]:
time_priors = [2, 4, 6, 8]
time_duration = 36
time_sep = 24
noDataPoints = 90

In [3]:
chunks=pd.read_csv('vitalPeriodic.csv', chunksize=1000000)

In [4]:
def gety_fun(pos, time_prior, data_frame):
    tmp = data_frame.iloc[-1]['temperature']      
        
    data_frame = data_frame.iloc[:-pos-1][['sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']]
    
    return data_frame.iloc[len(data_frame)-noDataPoints:-1].values, time_prior

In [5]:
def process(demo_df):
    demo_df = demo_df[['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']]
    demo_df[['vitalperiodicid','patientunitstayid', 'observationoffset']] = demo_df[['vitalperiodicid','patientunitstayid', 'observationoffset']].astype('int32')
    demo_df[['temperature','sao2', 'heartrate', 'respiration','cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']] = demo_df[['temperature','sao2', 'heartrate', 'respiration','cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']].astype('float32')
    
  #  demo_df['vitalperiodicid'] = demo_df['vitalperiodicid'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    demo_df['temperature'] = demo_df['temperature'].apply(lambda x: np.nan if x<38.0 else x)
    
     
    temp = demo_df.dropna(subset=['temperature'])[['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']]
    
    pospid = temp.patientunitstayid.unique()
    
    volume = len(temp.patientunitstayid.unique())
    #Removing those pid who does not have any fever data.
    demo_df = demo_df.drop(demo_df[[volume == Sum for Sum in sum([demo_df.patientunitstayid!=pid for pid in temp.patientunitstayid.unique()])]].index)
    
    dt = {}
    colms = ['systemicsystolic', 'systemicdiastolic', 'systemicmean', 'sao2', 'heartrate', 'respiration', 'cvp']
    for pid in demo_df.patientunitstayid.unique():
        for col in colms:
            dt[(pid, col)] = demo_df[demo_df['patientunitstayid']==pid][col].median()
    for pid in temp.patientunitstayid.unique():
        for col in colms:
            temp.loc[(temp.patientunitstayid==pid) & (temp[col].isnull()), col] = dt[(pid, col)]
    temp['systemicdiastolic'] = temp['systemicdiastolic'].fillna(demo_df['systemicdiastolic'].median())
    temp['systemicsystolic'] = temp['systemicsystolic'].fillna(demo_df['systemicsystolic'].median())
    temp['systemicmean'] = temp['systemicmean'].fillna(demo_df['systemicmean'].median())
    temp['sao2'] = temp['sao2'].fillna(demo_df['sao2'].median())
    temp['heartrate'] = temp['heartrate'].fillna(demo_df['heartrate'].median())
    temp['respiration'] = temp['respiration'].fillna(demo_df['respiration'].median())
    temp['cvp'] = temp['cvp'].fillna(demo_df['cvp'].median())
    
    for pid in demo_df.patientunitstayid.unique():
        for col in colms:
            demo_df.loc[(demo_df.patientunitstayid==pid) & (demo_df[col].isnull()), col] = dt[(pid, col)]

    demo_df['systemicdiastolic'] = demo_df['systemicdiastolic'].fillna(demo_df['systemicdiastolic'].median())
    demo_df['systemicsystolic'] = demo_df['systemicsystolic'].fillna(demo_df['systemicsystolic'].median())
    demo_df['systemicmean'] = demo_df['systemicmean'].fillna(demo_df['systemicmean'].median())
    demo_df['sao2'] = demo_df['sao2'].fillna(demo_df['sao2'].median())
    demo_df['heartrate'] = demo_df['heartrate'].fillna(demo_df['heartrate'].median())
    demo_df['respiration'] = demo_df['respiration'].fillna(demo_df['respiration'].median())
    demo_df['cvp'] = demo_df['cvp'].fillna(demo_df['cvp'].median())
    
    col_names =  demo_df.columns
    sorted_df  = pd.DataFrame(columns = col_names)
    for pid in demo_df.patientunitstayid.unique():
        sorted_df = sorted_df.append(demo_df[demo_df.patientunitstayid==pid].sort_values(by='observationoffset'))
    col_names =  temp.columns
    df  = pd.DataFrame(columns = col_names)
    for pid in temp.patientunitstayid.unique():
        df = df.append(temp[temp.patientunitstayid==pid].sort_values(by='observationoffset'))
    sorted_df.reset_index(drop=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    df_epCount  = pd.DataFrame(columns = ['epCount'])
    for pid in sorted_df.patientunitstayid.unique():
       # print(pid)
        df_fever = sorted_df[sorted_df.patientunitstayid==pid]['temperature'].apply(lambda x: 1 if x>=38.0 else 0)        
        df_epCount = pd.concat([df_epCount, (df_fever.cumsum()-df_fever).apply(lambda x: str(x))])
    
    sorted_df['patientunitstayid'] = sorted_df['patientunitstayid'].apply( lambda x: str(x))
    sorted_df['epCount'] = df_epCount[0]
    sorted_df['patientunitstayid'] =  sorted_df[['patientunitstayid', 'epCount']].apply(lambda x: '_'.join(x), axis=1)
    colm = ['sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']
    
    
    x=[]
    y=[]
    z=[]
    for clmn in colm:
        mean = sorted_df[clmn].mean()
        std = sorted_df[clmn].std()
        sorted_df[clmn] = sorted_df[clmn].apply(lambda x: (x-mean)/std)
    
   
        
    temperature_list = sorted_df[sorted_df['temperature'].notnull()].index.tolist()
    i=0
    prev_idx = 0
    pid = sorted_df.iloc[0]['patientunitstayid']
    lastFever = sorted_df.iloc[0]['observationoffset']# does not matter
    prev_pid = -1
    

    for idx in temperature_list:   
        pid = sorted_df.iloc[idx]['patientunitstayid']
        if not pid==prev_pid:
            if not prev_pid==-1:
                lastFever = sorted_df[sorted_df['patientunitstayid'] == prev_pid].iloc[-1]['observationoffset']
            prev_pid = sorted_df.iloc[idx]['patientunitstayid']
            prev_idx = sorted_df.index[sorted_df['patientunitstayid'] == pid].tolist()[0]
            
        if sorted_df.loc[idx]['epCount'] == '0' or sorted_df.loc[idx]['observationoffset'] - lastFever > time_sep*60:

        
            while sorted_df.iloc[idx]['observationoffset'] - sorted_df.iloc[prev_idx]['observationoffset'] > time_duration*60:
                prev_idx+=1
 #   print(prev_idx, idx)
            time_diff = sorted_df.loc[idx]['observationoffset'] - sorted_df.loc[prev_idx]['observationoffset']
    
  #  print(time_diff)
            for time_prior in time_priors:
                if time_diff < time_prior*60:
                    pass
    
                else:
               # freq = len(sorted_df[prev_idx:idx+1])/time_diff.total_seconds()
                    freq = 1/5
                    pos = int(60*time_prior*freq)
                    if len(sorted_df[prev_idx:idx+1].iloc[:-pos-1])<=noDataPoints:
                        pass
                    else:
                        oop = gety_fun(pos, time_prior, sorted_df[prev_idx:idx+1])
                        x.append(oop[0])
                        y.append(oop[1])
                        z.append(pid)
        i+=1
    
    y = np.array(y)
    x=np.array(x)
    return y, x, z, pospid

In [6]:

iu = 0
Y , X, Z = [], [], []
pospid = []
for chunk in chunks:
    iu+=1
    y, x, z, pid = process(chunk)
    pospid.append(pid)
    print(iu)
    print(len(pid), sum(y), len(y))
    for i in range(len(y)):
        Y.append(y[i])
        X.append(x[i])
        Z.append(z[i])


/home/aditya/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/aditya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


1
129 2022 415


/home/aditya/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/aditya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


2
112 2342 475


/home/aditya/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/aditya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


...

145
29 176 37


/home/aditya/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/aditya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


146
38 460 96


/home/aditya/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/aditya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


147
25 258 54


In [12]:
np.save('TCNRegressionobshalfX', X)
np.save('TCNRegressionobshalfY', Y)
np.save('TCNRegressionobshalfZ', Z)

In [2]:

X = np.load('TCNRegressionobshalfX.npy')
Y = np.load('TCNRegressionobshalfY.npy')


In [3]:
X.shape

(28459, 89, 7)

In [5]:
'''
tempX = np.load('TCNRegressionobsconX.npy')
tempY = 1+np.load('TCNRegressionobsconY.npy')

sz = len(tempY)

Xall = list(X)
Yall = list(np.asarray(list(Y/2)).astype(np.int32))

for i in range(sz):
    pos = np.random.randint(0, sz)
    Xall.insert(pos, list(tempX[i]))
    Yall.insert(pos, tempY[i])
    
Xall = np.asarray(Xall)

Yall = np.asarray(Yall)
'''

In [6]:
#Yall.shape, Xall.shape

((95817,), (95817, 89, 7))

In [16]:
sum((Y==8).astype(int))

6809

In [20]:
int(list(np.asarray(Y)/2))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [11]:
Yall[:500]

array([1, 2, 3, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 3, 0, 4, 1, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 0, 4, 1, 0, 0, 2, 3, 0, 0, 0, 4, 0, 0, 1,
       0, 2, 0, 3, 0, 4, 0, 0, 0, 1, 0, 0, 0, 2, 3, 0, 0, 4, 1, 0, 2, 3,
       0, 0, 0, 4, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 4, 1, 2, 3,
       0, 4, 0, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 4, 1, 2, 0, 0, 0, 0, 3, 4,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 4, 1, 0, 2, 0, 0,
       0, 3, 0, 0, 0, 4, 0, 1, 2, 3, 4, 0, 1, 0, 2, 3, 0, 4, 1, 2, 0, 0,
       0, 3, 4, 1, 0, 2, 0, 3, 4, 1, 0, 0, 0, 0, 2, 3, 0, 4, 0, 0, 0, 1,
       0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 4, 0, 1, 2, 3, 0, 0, 0, 4, 0, 0, 1, 0,
       2, 0, 0, 3, 4, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4,
       0, 0, 1, 2, 3, 0, 0, 0, 4, 1, 2, 3, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 2, 0, 0, 3, 4, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 3, 4,
       0, 0, 0, 1, 0, 2, 0, 0, 3, 0, 0, 0, 0, 4, 1,

In [12]:
_Y = np.eye(4)[(np.asarray(Y)/2-1).astype(np.int32)]

In [13]:
_Y

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [21]:
#_Y=(np.asarray(Y)/2-1).astype(np.int32)

In [4]:
len(Xall)

NameError: name 'Xall' is not defined

In [ ]:
batch_size, timesteps, input_dim = None, 89, 7

i = Input(batch_shape=(batch_size, timesteps, input_dim))

o = TCN(kernel_size=4, nb_stacks=2, dilations=[1, 2, 4, 8, 16, 32], return_sequences=False)(i)  # The TCN layers are here.
o = Dense(64, activation='relu')(o)
#o = Dropout(0.2)(o)
o = Dense(4, activation='softmax')(o)
m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Y = np.array(_Y)
#X = np.array(X)
temp, X_cv, label, y_cv = train_test_split(Xall, _Y, test_size=0.2, random_state=2)

m.fit(temp, label, epochs=80, validation_data=(X_cv, y_cv))



Train on 76653 samples, validate on 19164 samples
Epoch 1/80
76653/76653 [==============================] - 252s 3ms/step - loss: 11.7516 - acc: 0.0782 - val_loss: 11.7306 - val_acc: 0.0792
Epoch 2/80
76653/76653 [==============================] - 269s 4ms/step - loss: 11.7453 - acc: 0.0813 - val_loss: 11.7301 - val_acc: 0.0812
Epoch 3/80
76653/76653 [==============================] - 271s 4ms/step - loss: 11.7450 - acc: 0.0817 - val_loss: 11.7309 - val_acc: 0.0777
Epoch 4/80
76653/76653 [==============================] - 267s 3ms/step - loss: 13.2007 - acc: 0.0807 - val_loss: 14.8363 - val_acc: 0.0795
Epoch 5/80
76653/76653 [==============================] - 247s 3ms/step - loss: 14.8251 - acc: 0.0802 - val_loss: 14.8363 - val_acc: 0.0795
Epoch 6/80
76653/76653 [==============================] - 291s 4ms/step - loss: 14.8251 - acc: 0.0802 - val_loss: 14.8363 - val_acc: 0.0795
Epoch 7/80
76653/76653 [==============================] - 238s 3ms/step - loss: 14.8251 - acc: 0.0802 - val_lo

In [8]:
m.fit(temp, label, epochs=80, validation_data=(X_cv, y_cv))

Train on 22767 samples, validate on 5692 samples
Epoch 1/80
22767/22767 [==============================] - 73s 3ms/step - loss: 0.1714 - acc: 0.9429 - val_loss: 1.1506 - val_acc: 0.7096
Epoch 2/80
22767/22767 [==============================] - 74s 3ms/step - loss: 0.1804 - acc: 0.9385 - val_loss: 1.1891 - val_acc: 0.7205
Epoch 3/80
 2688/22767 [==>...........................] - ETA: 1:01 - loss: 0.2065 - acc: 0.9379

KeyboardInterrupt: 

In [10]:
opa = np.argmax( m.predict(X_cv), axis=1)
k = 0
po = np.argmax(y_cv, axis=1)
for i in range(len(opa)):
    k+=int(po[i]==opa[i])
    if po[i]!=opa[i]:
        print(po[i], opa[i])
print(k, len(opa))

1 0
3 0
2 1
0 1
0 1
3 2
3 2
3 0
1 0
3 0
1 0
0 3
3 0
0 3
3 0
3 0
1 2
3 1
0 1
1 0
3 2
2 0
2 1
1 3
0 2
3 1
0 1
3 0
0 3
2 3
3 0
1 2
3 0
0 3
3 0
1 3
3 2
3 0
3 1
0 2
0 1
2 1
2 0
1 0
0 1
2 0
0 3
0 3
1 0
2 1
3 0
0 3
1 3
0 3
1 3
1 3
2 1
2 0
3 1
0 1
2 3
2 1
1 2
1 3
0 2
1 2
3 0
2 1
0 3
3 0
3 0
0 2
1 3
3 2
2 1
0 3
0 3
3 0
0 3
2 1
2 1
0 1
0 3
3 0
2 3
2 1
0 3
0 1
1 3
2 0
3 0
3 0
3 1
0 2
1 0
3 0
3 0
2 3
0 2
0 3
2 3
0 1
0 2
2 1
0 1
0 2
2 0
2 3
3 0
1 2
1 3
3 0
2 0
3 1
2 0
1 2
0 1
3 0
3 0
2 3
0 1
0 3
3 0
2 1
3 2
3 0
0 2
0 2
0 1
2 0
0 2
0 2
2 1
0 1
0 1
1 3
0 2
0 1
0 3
3 0
0 3
3 0
0 3
2 0
1 2
1 2
0 2
1 0
3 2
3 0
2 0
3 1
1 0
3 0
3 0
1 0
3 0
3 2
1 3
3 2
0 1
1 0
0 1
2 1
2 1
0 1
0 3
2 1
3 1
0 2
3 0
0 1
3 0
0 3
1 2
3 2
3 0
0 1
3 0
3 0
0 1
0 3
0 3
3 1
3 0
3 1
0 2
1 0
0 1
3 1
2 1
3 1
2 1
2 3
0 3
2 3
3 0
2 3
0 1
0 3
1 3
3 0
3 0
0 2
1 3
3 2
3 0
0 2
3 1
3 0
3 1
2 3
3 1
3 1
2 0
3 0
3 1
0 3
3 1
2 1
0 2
3 2
1 3
3 1
0 2
0 2
1 0
0 1
0 1
0 2
3 1
3 0
3 1
2 0
3 0
3 0
0 3
1 0
3 0
3 1
2 1
0 1
0 1
0 3
0 3
0 2
3 0
0 3
0 1
0 3


In [11]:
m.save('halfoverfitting2055.h5')

In [5]:
from keras.layers import LSTM

batch_size, timesteps, input_dim = None, 89, 7

i = Input(batch_shape=(batch_size, timesteps, input_dim))

#o = TCN(kernel_size=2, nb_stacks=2, dilations=[1, 2, 4, 8, 16, 32], return_sequences=True)(i)  # The TCN layers are here.
o = LSTM(128, return_sequences=False)(i)
o = Dense(32, activation='relu')(o)
o = Dense(4, activation='softmax')(o)
m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

temp, X_cv, label, y_cv = train_test_split(X, _Y, test_size=0.2, random_state=2)



m.fit(temp, label, epochs=80, validation_data=(X_cv, y_cv))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 22767 samples, validate on 5692 samples
Epoch 1/80
22767/22767 [==============================] - 111s 5ms/step - loss: 1.3869 - acc: 0.2688 - val_loss: 1.3857 - val_acc: 0.2730
Epoch 2/80
22767/22767 [==============================] - 102s 4ms/step - loss: 1.3830 - acc: 0.2779 - val_loss: 1.3832 - val_acc: 0.2686
Epoch 3/80
22767/22767 [==============================] - 104s 5ms/step - loss: 1.3817 - acc: 0.2796 - val_loss: 1.3855 - val_acc: 0.2772
Epoch 4/80
22767/22767 [==============================] - 109s 5ms/step - loss: 1.3818 - acc: 0.2831 - val_loss: 1.3840 - val_acc: 0.2704
Epoch 5/80
22767/22767 [==============================] - 106s 5ms/step - loss: 1.3811 - acc: 0.2787 - val_loss: 1.3845 - val_acc: 0.2674
Epoch 6/80
22767/22767 [==============================] - 106s 5ms/step - loss: 1.3811 - acc: 0.2806 - val_loss: 1.3839 - val_acc: 0.2690
Epo

KeyboardInterrupt: 

In [27]:
m.predict(X)

array([[9.7109580e-01, 4.4328647e-04, 2.1976562e-02, 6.4844447e-03],
       [9.5734268e-01, 1.3204853e-02, 8.0531193e-03, 2.1399336e-02],
       [5.2113825e-07, 1.8392863e-03, 9.4985807e-01, 4.8302077e-02],
       ...,
       [8.8492286e-04, 9.9896693e-01, 4.4662840e-05, 1.0349542e-04],
       [7.8478479e-05, 2.5720070e-03, 9.9726784e-01, 8.1697050e-05],
       [2.4707275e-04, 1.8044915e-04, 1.2030815e-02, 9.8754162e-01]],
      dtype=float32)

In [17]:
m.fit(temp, label, epochs=40, validation_data=(X_cv, y_cv))

Train on 22767 samples, validate on 5692 samples
Epoch 1/40
22767/22767 [==============================] - 58s 3ms/step - loss: 0.1760 - acc: 0.9400 - val_loss: 1.1636 - val_acc: 0.6985
Epoch 2/40
22767/22767 [==============================] - 59s 3ms/step - loss: 0.2173 - acc: 0.9274 - val_loss: 1.1670 - val_acc: 0.7006
Epoch 3/40
22767/22767 [==============================] - 60s 3ms/step - loss: 0.1719 - acc: 0.9439 - val_loss: 1.1803 - val_acc: 0.7034
Epoch 4/40
22767/22767 [==============================] - 60s 3ms/step - loss: 0.1787 - acc: 0.9418 - val_loss: 1.3288 - val_acc: 0.7080
Epoch 5/40
22767/22767 [==============================] - 60s 3ms/step - loss: 0.2143 - acc: 0.9291 - val_loss: 1.2781 - val_acc: 0.6903
Epoch 6/40
22767/22767 [==============================] - 64s 3ms/step - loss: 0.1673 - acc: 0.9430 - val_loss: 1.2261 - val_acc: 0.7177
Epoch 7/40
22767/22767 [==============================] - 60s 3ms/step - loss: 0.1594 - acc: 0.9465 - val_loss: 1.3715 - val_acc:

KeyboardInterrupt: 

In [29]:
y_cv

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]])

In [30]:
 m.predict(X_cv)

array([[4.5196532e-13, 1.0000000e+00, 4.3555662e-15, 2.7318295e-13],
       [2.1369487e-01, 4.0132928e-04, 7.8194886e-01, 3.9549856e-03],
       [9.9986064e-01, 1.1027115e-08, 1.4171957e-08, 1.3933562e-04],
       ...,
       [5.6853466e-02, 5.7987929e-03, 5.4398514e-03, 9.3190789e-01],
       [1.5524886e-19, 2.4722350e-19, 1.0000000e+00, 4.4397207e-18],
       [9.9449205e-01, 8.2988163e-06, 1.0723788e-05, 5.4889224e-03]],
      dtype=float32)

In [9]:
prd = m.predict(X_cv)


In [32]:
opa = np.argmax( m.predict(X_cv), axis=1)
k = 0
po = np.argmax(y_cv, axis=1)
for i in range(len(opa)):
    k+=int(po[i]==opa[i])
k, len(opa)

In [33]:
opa

array([1, 2, 0, ..., 3, 2, 0])

In [34]:
np.argmax(y_cv, axis=1)

array([1, 2, 0, ..., 3, 2, 0])

In [35]:
k = 0
po = np.argmax(y_cv, axis=1)
for i in range(len(opa)):
    k+=int(po[i]==opa[i])

In [37]:
k, len(opa)

(2078, 2846)